# Filtering

Nornir doesn't have the concept of "roles" or "classes" or anything like that. Instead, it provides different mechanisms for you to classify and filter hosts so you can implement the mechanism that fits your workflow better.

Those three mechanisms are:

1. Simple filtering
2. Advanced filtering
3. Functions

Let's start by initializing nornir.

In [1]:
from nornir import InitNornir
nr = InitNornir(
    inventory={
        "options": {
            "host_file": "3_filtering/inventory/hosts.yaml",
            "group_file": "3_filtering/inventory/groups.yaml",
            "defaults_file": "3_filtering/inventory/defaults.yaml",
        }
    }
)

## Simple filtering

It allows filtering by matching on `<k,v>` pairs. It's the simplest way of filtering hosts, however, it is not very flexible. 

In [2]:
# role and region are user-defined data
leafs = nr.filter(role="leaf")
for host, host_data in leafs.inventory.hosts.items():
    print(f"{host}: {host_data['role']}")

leaf00.bma: leaf
leaf01.bma: leaf
leaf00.cmh: leaf
leaf01.cmh: leaf


In [3]:
leafs_eu = nr.filter(role="leaf", region="EU")
for host, host_data in leafs_eu.inventory.hosts.items():
    print(f"{host}: {host_data['role']}, {host_data['region']}")

leaf00.bma: leaf, EU
leaf01.bma: leaf, EU


In [4]:
# filtering is additive
my_devs = nr.filter(role="leaf")
my_devs = my_devs.filter(region="EU")
for host, host_data in my_devs.inventory.hosts.items():
    print(f"{host}: {host_data['role']}, {host_data['region']}")

leaf00.bma: leaf, EU
leaf01.bma: leaf, EU


In [5]:
# you can also filter using well-known attributes
junos_devs = nr.filter(platform="junos")
for host, host_data in junos_devs.inventory.hosts.items():
    print(f"{host}: {host_data.platform}")

spine00.bma: junos
leaf00.bma: junos
spine00.cmh: junos
leaf00.cmh: junos


## Advanced Filtering

Extremely powerful and flexible, however, it's not as straightforward.

First, you will need to import the ``F`` object:

In [6]:
from nornir.core.filter import F

Let's start by porting the examples in "simple filtering". 

In [7]:
# role and region are user-defined data
leafs = nr.filter(F(role="leaf"))
for host, host_data in leafs.inventory.hosts.items():
    print(f"{host}: {host_data['role']}")

leaf00.bma: leaf
leaf01.bma: leaf
leaf00.cmh: leaf
leaf01.cmh: leaf


In [8]:
leafs_eu = nr.filter(F(role="leaf") & F(region="EU"))
for host, host_data in leafs_eu.inventory.hosts.items():
    print(f"{host}: {host_data['role']}, {host_data['region']}")

leaf00.bma: leaf, EU
leaf01.bma: leaf, EU


In [9]:
# filtering is additive
my_devs = nr.filter(F(role="leaf"))
my_devs = my_devs.filter(F(region="EU"))
for host, host_data in my_devs.inventory.hosts.items():
    print(f"{host}: {host_data['role']}, {host_data['region']}")

leaf00.bma: leaf, EU
leaf01.bma: leaf, EU


In [10]:
# you can also filter using well-known attributes
junos_devs = nr.filter(F(platform="junos"))
for host, host_data in junos_devs.inventory.hosts.items():
    print(f"{host}: {host_data.platform}")

spine00.bma: junos
leaf00.bma: junos
spine00.cmh: junos
leaf00.cmh: junos


``F`` objects support logical operations:

In [11]:
# leafs in the EU
leafs_eu = nr.filter(F(role="leaf") & F(region="EU"))
for host, host_data in leafs_eu.inventory.hosts.items():
    print(f"{host}: {host_data['role']}, {host_data['region']}")

leaf00.bma: leaf, EU
leaf01.bma: leaf, EU


In [12]:
# leaves NOT in the EU
leafs_not_eu = nr.filter(F(role="leaf") & ~F(region="EU"))
for host, host_data in leafs_not_eu.inventory.hosts.items():
    print(f"{host}: {host_data['role']}, {host_data['region']}")

leaf00.cmh: leaf, NA
leaf01.cmh: leaf, NA


In [13]:
# either ios or eos switches
ios_or_eos = nr.filter(F(platform="ios") | F(platform="eos"))
for host, host_data in ios_or_eos.inventory.hosts.items():
    print(f"{host}: {host_data.platform}")

spine01.bma: eos
leaf01.bma: eos
spine01.cmh: ios
leaf01.cmh: ios


In [14]:
# junos acting as leaves located outside the EU
junos_leaves_not_in_eu = nr.filter(F(platform="junos") & F(role="leaf") & ~F(region="EU"))
for host, host_data in junos_leaves_not_in_eu.inventory.hosts.items():
    print(f"{host}: {host_data.platform}, {host_data['role']}, {host_data['region']}")

leaf00.cmh: junos, leaf, NA


The F objects also let's you filter by nested data

In [15]:
#  we have added similar information to all the hosts to represent with image they are running
!sed '10,18!d' 3_filtering/inventory/hosts.yaml


    data:  # user-defined data
        site: bma
        role: spine
        region: EU
        system:
            image: 17.2R2   # image version
            uptime: 100     # in days, somehow the inventory knows this



In [16]:
junos_image = nr.filter(F(platform="junos") & F(system__image="14.1X53-D46"))
for host, host_data in junos_image.inventory.hosts.items():
    print(f"{host}: {host_data.platform}, {host_data['system']['image']}")

leaf00.bma: junos, 14.1X53-D46
leaf00.cmh: junos, 14.1X53-D46


The `F` object also lets you perform certain operations on objects:

In [17]:
junos_17 = nr.filter(F(platform="junos") & F(system__image__startswith="17"))
for host, host_data in junos_17.inventory.hosts.items():
    print(f"{host}: {host_data.platform}, {host_data['system']['image']}")

spine00.bma: junos, 17.2R2
spine00.cmh: junos, 17.2R1


In [18]:
such_uptime = nr.filter(F(system__uptime__ge=50))
for host, host_data in such_uptime.inventory.hosts.items():
    print(f"{host}: {host_data['system']['uptime']}")

spine00.bma: 100
spine01.bma: 100
spine01.cmh: 50
leaf00.cmh: 60


In [19]:
various_images = nr.filter(F(platform="junos") & F(system__image__any=["14.1X53-D46", "17.2R1"]))
for host, host_data in various_images.inventory.hosts.items():
    print(f"{host}: {host_data.platform}, {host_data['system']['image']}")

leaf00.bma: junos, 14.1X53-D46
spine00.cmh: junos, 17.2R1
leaf00.cmh: junos, 14.1X53-D46


## Functions

You can also filter objects by providing your own function. Such function will accept a `host` as parameter and needs to return either `True` (accept)  or `False` (filter).

In [20]:
import re

def eos_maintenance_images(host):
    """ EOS images end either in M (maintenance images) or F (feature images)"""
    return bool(re.match(".*M$", host["system"]["image"]))

In [21]:
maintenance_images = nr.filter(filter_func=eos_maintenance_images)
for host, host_data in maintenance_images.inventory.hosts.items():
    print(f"{host}: {host_data.platform}, {host_data['system']['image']}")

spine01.bma: eos, 4.19.4M
leaf01.bma: eos, 4.17.4M
